# Structural Deep Network Embedding

-  参考深度之眼赵老师

In [1]:
import torch
import torch.optim as optim
from argparse import ArgumentParser, ArgumentDefaultsHelpFormatter
from torch.utils.data.dataloader import DataLoader
import utils
from data import dataset
from models import model
import numpy as np

In [3]:
def parse_args():
    # 使用parser加载信息
    parser = ArgumentParser(formatter_class=ArgumentDefaultsHelpFormatter,
                            conflict_handler='resolve')
    # 输入文件
    parser.add_argument('--input', default='./data/cora/cora_edgelist.txt',
                        help='Input graph file')
    # 训练结果embedding输出文件
    parser.add_argument('--output', default='./data/cora/Vec.emb',
                        help='Output representation file')
    # 并行参数
    parser.add_argument('--workers', default=8, type=int,
                        help='Number of parallel processes.')
    # 处理有/无权图
    parser.add_argument('--weighted', action='store_true', default=False,
                        help='Treat graph as weighted')
    # epoch数量
    parser.add_argument('--epochs', default=100, type=int,
                        help='The training epochs of SDNE')
    # dropout比例值
    parser.add_argument('--dropout', default=0.5, type=float,
                        help='Dropout rate (1 - keep probability)')
    
    parser.add_argument('--weight-decay', type=float, default=5e-4,
                        help='Weight for L2 loss on embedding matrix')
    # 学习率设置
    parser.add_argument('--lr', default=0.001, type=float,
                        help='learning rate')
    # 模型参数，一阶相似度和二阶相似度之间的比重
    parser.add_argument('--alpha', default=1e-2, type=float,
                        help='alhpa is a hyperparameter in SDNE')
    # 图的稀疏性问题，论文中的参数beta, 侧重学习邻接矩阵中为1的值
    parser.add_argument('--beta', default=5., type=float,
                        help='beta is a hyperparameter in SDNE')
    
    parser.add_argument('--nu1', default=1e-5, type=float,
                        help='nu1 is a hyperparameter in SDNE')
    
    parser.add_argument('--nu2', default=1e-4, type=float,
                        help='nu2 is a hyperparameter in SDNE')
    # batch大小
    parser.add_argument('--bs', default=100, type=int,
                        help='batch size of SDNE')
    # 自编码器第一层神经元个数
    parser.add_argument('--nhid0', default=1000, type=int,
                        help='The first dim')
    # 自编码器第二层神经元个数
    parser.add_argument('--nhid1', default=128, type=int,
                        help='The second dim')
    # 学习率步长设置
    parser.add_argument('--step_size', default=10, type=int,
                        help='The step size for lr')
    # 学习率gamma值设置
    parser.add_argument('--gamma', default=0.9, type=int,
                        help='The gamma for lr')
    args = parser.parse_args(args=[])

    return args

In [5]:
args = parse_args()

In [10]:
G, Adj, Node = Read_graph(args.input)

In [7]:
import networkx as nx
import numpy as np
from torch.utils import data
from torch.utils.data import DataLoader
import torch

In [8]:
def Read_graph(file_name):
    # 文本文件中的每一行必须含有相同的数据; delimiter分隔符默认是空格; 类型是numpy array
    edge = np.loadtxt(file_name).astype(np.int32)
    # 得到图中点的最小和最大编号; .min()返回数组中所有元素最小的
    min_node, max_node = edge.min(), edge.max()
    # Node表示图上一共有多少个顶点，如果标号是从0开始，那么顶点数 = max_node + 1
    if min_node == 0:
        Node = max_node + 1
    else:
        Node = max_node
    # 这里面使用networkx将图的信息存入
    G = nx.Graph()
    # Adj就是图的邻接表矩阵，是一个n*n大小的numpy矩阵，这里n是顶点的个数
    Adj = np.zeros([Node, Node], dtype=np.int32)
    # 遍历边的文件，将每条边存入networkx的图，以及邻接矩阵Adj所对应的位置(i, j)
    for i in range(edge.shape[0]):
        G.add_edge(edge[i][0], edge[i][1])
        if min_node == 0:
            Adj[edge[i][0], edge[i][1]] = 1
            Adj[edge[i][1], edge[i][0]] = 1
        else:
            Adj[edge[i][0] - 1, edge[i][1] - 1] = 1
            Adj[edge[i][1] - 1, edge[i][0] - 1] = 1
    # 转化成tensor
    Adj = torch.FloatTensor(Adj)
    return G, Adj, Node

In [9]:
# 继承自pytorch中的data.Dataset类，为了后面batch训练方便
class Dataload(data.Dataset):

    def __init__(self, Adj, Node):
        self.Adj = Adj
        self.Node = Node
    def __getitem__(self, index):
        return index
        # adj_batch = self.Adj[index]
        # adj_mat = adj_batch[index]
        # b_mat = torch.ones_like(adj_batch)
        # b_mat[adj_batch != 0] = self.Beta
        # return adj_batch, adj_mat, b_mat
    def __len__(self):
        return self.Node

In [13]:
 Node

2708

In [15]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

In [16]:
class MNN(nn.Module):
    def __init__(self, node_size, nhid0, nhid1, droput, alpha):
        # 声明神经网络都有哪些层
        # 这里的结构是：输入-encode0-encode1-decode0-decode1-输出
        
        # 于是如果我们输入的向量的长度为node_size，那么每一层神经元的个数分别是
        
        # 假设我们一次输入的batch_size = K
        # 输入 = [K, node_size]
        super(MNN, self).__init__()
        # encode0: [node_size, nhid0], 这一步输出 = [K, node_size] * [node_size, nhid0] = [K, nhid0]
        self.encode0 = nn.Linear(node_size, nhid0)
        # encode1: [nhid0, nhid1], 这一步输出 = [K, nhid0] * [nhid0, nhid1] = [K, nhid1]
        self.encode1 = nn.Linear(nhid0, nhid1)
        # decode0: [nhid1, nhid0], 这一步输出 = [K, nhid1] * [nhid1, nhid0] = [K, nhid0]
        self.decode0 = nn.Linear(nhid1, nhid0)
        # decode1: [nhid0, node_size], 这一步输出 = [K, nhid0] * [nhid0, node_size] = [K, node_size]
        # 自编码器希望最终的输出尽可能与输入相等
        self.decode1 = nn.Linear(nhid0, node_size)
        self.droput = droput
        # 论文公式(5)中的alpha
        self.alpha = alpha

    def forward(self, adj_batch, adj_mat, b_mat):
        # __init__中声明的神经网络的架构实现
        # 激活函数使用的是leaky_relu
        # 这里的结构是：输入-encode0-encode1-decode0-decode1-输出
        t0 = F.leaky_relu(self.encode0(adj_batch))
        t0 = F.leaky_relu(self.encode1(t0))
        # 这个中间结果使我们最终要保留的node embeeding
        embedding = t0
        t0 = F.leaky_relu(self.decode0(t0))
        t0 = F.leaky_relu(self.decode1(t0))
        embedding_norm = torch.sum(embedding * embedding, dim=1, keepdim=True)
        # 一阶相似度的loss值
        L_1st = torch.sum(adj_mat * (embedding_norm -
                                     2 * torch.mm(embedding, torch.transpose(embedding, dim0=0, dim1=1))
                                     + torch.transpose(embedding_norm, dim0=0, dim1=1)))
        # 二阶相似度的loss值
        # adj_batch是论文中的input X, t0是论文中最后输出的X^
        L_2nd = torch.sum(((adj_batch - t0) * b_mat) * ((adj_batch - t0) * b_mat))
        return L_1st, self.alpha * L_2nd, L_1st + self.alpha * L_2nd

    def savector(self, adj):
        t0 = self.encode0(adj)
        t0 = self.encode1(t0)
        return t0
    
# broadcast

In [ ]:
# ./models/model.py中的MNN类，基于pytorch实现的论文中的自编码器
model = model.MNN(Node, args.nhid0, args.nhid1, args.dropout, args.alpha)
# Adam算法优化模型参数
opt = optim.Adam(model.parameters(), lr=args.lr)
# 设置模型的学习率的超参数
scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=args.step_size, gamma=args.gamma)
Data = dataset.Dataload(Adj, Node)
# 按batchsize将训练样本分组
Data = DataLoader(Data, batch_size=args.bs, shuffle=True, )